In [1]:
import os
import CityHub
#import folium
import geopandas as gpd
import pandas as pd

import math
from statistics import mean

In [2]:
ch = CityHub.CityHub('São Paulo, São Paulo, Brazil')

Loading file...
Preprocessing city mesh...
Refining mesh...


In [3]:
G = ch.city_street_graph

In [4]:
ch.load_PALayer_mesh('data/sp/sp_setores_censitarios/35SEE250GC_SIR.shp','CD_GEOCODI',40.0)

Refining mesh...


0

In [5]:
ch.load_PALayers_csv_data(0,'data/sp/census_data/data_preprocess.csv','Cod_setor')

True

In [6]:
ch.load_PBLayer('data/sp/infraestrutura/SAD69-96_SHP_pontoonibus/SAD69-96_SHP_pontoonibus.shp',23,'K',True,-45.0,-45.0)

0

In [7]:
ch.load_PBLayer('data/sp/infraestrutura/SAD69-96_SHP_estacaometro/SAD69-96_SHP_estacaometro_point.shp',23,'K')

1

In [8]:
ch.load_PBLayer('data/sp/infraestrutura/SAD69-96_SHP_estacaotrem/SAD69-96_SHP_estacaotrem.shp',23,'K')

2

In [9]:
ch.load_PBLayer('data/sp/infraestrutura/terminal_onibus/sad6996_terminal_onibus.shp',23,'K')

3

In [10]:
from shapely.geometry import Polygon, Point

city_vert_gdf = gpd.GeoDataFrame(list(range(0,len(ch.city_vert_list))),geometry = gpd.points_from_xy(pd.DataFrame(ch.city_vert_list)[0], pd.DataFrame(ch.city_vert_list)[1]))
city_vert_gdf.columns = ["vert", "geometry"]
city_vert_gdf

,vert,geometry
0,0,POINT (-23.56299 -46.70397)
1,1,POINT (-23.56746 -46.70223)
2,2,POINT (-23.57296 -46.69923)
3,3,POINT (-23.57183 -46.70207)
4,4,POINT (-23.53462 -46.74815)
...,...,...
121537,121537,POINT (-23.55463 -46.53176)
121538,121538,POINT (-23.57349 -46.49451)
121539,121539,POINT (-23.57281 -46.49286)
121540,121540,POINT (-23.56551 -46.48308)


In [11]:
def compute_static_feature_vector(vert_ind):

    feature_vec = pd.Series(dtype=float)

    layer_pontodeonibus = 0
    layer_estacaodemetro = 1
    layer_estacaodetrem = 2
    layer_terminaldeonibus = 3

    query_coords=ch.city_vert_list[vert_ind]

#       transporte
    transporte_indices = ['Pontos_de_onibus','Estacao_de_metro', 'Estacao_de_trem', 'Terminal_de_onibus']
    transporte_list = pd.Series([0.0]*len(transporte_indices), index = transporte_indices)
    transporte_list['Pontos_de_onibus'] = len(ch.query_point_in_PBLayer(0,query_coords[0],query_coords[1],layer_pontodeonibus,0.2))
    transporte_list['Estacao_de_metro'] = len(ch.query_point_in_PBLayer(1,query_coords[0],query_coords[1],layer_estacaodemetro,0.2))
    transporte_list['Estacao_de_trem'] = len(ch.query_point_in_PBLayer(2,query_coords[0],query_coords[1],layer_estacaodetrem,0.2))
    transporte_list['Terminal_de_onibus'] = len(ch.query_point_in_PBLayer(3,query_coords[0],query_coords[1],layer_terminaldeonibus,0.2))

    feature_vec = pd.concat([feature_vec, transporte_list])

#       favelas
#    favela_indices = ['Favela_proxima']
#    favela_list = pd.Series([0.0]*len(favela_indices), index = favela_indices)
#    favela_list['Favela_proxima'] = (len(ch.query_point_in_RDLayer(0, query_coords[0],query_coords[1],0.5,True))>0)*1

#    feature_vec = pd.concat([feature_vec, favela_list])

    return feature_vec

In [12]:
X_static = city_vert_gdf["vert"].apply(lambda vert: compute_static_feature_vector(vert))

In [13]:
X_static

,Pontos_de_onibus,Estacao_de_metro,Estacao_de_trem,Terminal_de_onibus
0,0.0,1.0,3.0,1.0
1,0.0,3.0,3.0,1.0
2,0.0,3.0,3.0,1.0
3,0.0,2.0,4.0,1.0
4,0.0,0.0,3.0,0.0
...,...,...,...,...
121537,0.0,0.0,0.0,1.0
121538,0.0,0.0,0.0,0.0
121539,0.0,0.0,0.0,0.0
121540,0.0,0.0,0.0,0.0


In [45]:
city_vert_gdf.iloc[0]

vert                                      0
geometry    POINT (-23.5629908 -46.7039651)
Name: 0, dtype: object

In [47]:
ch.city_vert_list[0]

(-23.5629908, -46.7039651)

In [60]:
ch.query_point_in_PBLayer(0,ch.city_vert_list[0][0],ch.city_vert_list[0][1],0)

[]

In [61]:
ch.query_point_in_PBLayer(1,ch.city_vert_list[0][0],ch.city_vert_list[0][1],1)

[(-23.56677543060003, -46.701248656863875)]

In [62]:
ch.query_point_in_PBLayer(2,ch.city_vert_list[0][0],ch.city_vert_list[0][1],2)

[(-23.56605954681605, -46.70259752571509),
 (-23.55723463242395, -46.71133035159957),
 (-23.573203731944503, -46.69834557655026)]

In [63]:
ch.query_point_in_PBLayer(3,ch.city_vert_list[0][0],ch.city_vert_list[0][1],3)

[(-23.56586157048122, -46.70130560104628)]

In [51]:
city_vert_gdf["vert"].apply(lambda vert: vert * -1)

0              0
1             -1
2             -2
3             -3
4             -4
           ...  
121537   -121537
121538   -121538
121539   -121539
121540   -121540
121541   -121541
Name: vert, Length: 121542, dtype: int64